In [19]:
# This file aims at converting units that are not grammes in grammes
# we consider here that : 1 c.à.c = 5 g and 1 c.à.s = 15 grammes

In [1]:
!pip install lxml 

In [5]:
from scrapping import *
import re

In [200]:
url_test = "https://www.marmiton.org/recettes/recette_moelleux-au-chocolat-sans-beurre-sans-sucre_14748.aspx"

test=find_all_recipes("Steak Tartare")



100%|██████████| 15/15 [00:03<00:00,  3.83it/s]


In [92]:
test.items()

dict_items([('Steak tartare', {'moutarde de Dijon': '1c.à.c', 'oignon': '1c.à.s', 'câpres': '1c.à.s', 'sauce worcestershire': '1c.à.c', 'ketchup': '1c.à.s', 'tabasco': '', 'poivre': '', 'sel': '', "huile d'olive": '2c.à.s', 'persil': '1c.à.c', 'boeuf': '250g', "jaune d'oeuf": '1'}), ('Steak tartare arrangé', {'huile': '450g', 'moutarde': '', 'ketchup': '', 'tabasco': '', 'poivre': '', 'sel': '', 'câpres': '', 'persil': '', 'boeuf haché': '', 'oeuf': '1', 'échalote': '1', 'cornichons': ''}), ('Steak tartare végétarien', {'carotte': '500g', 'câpres': '2c.à.s', 'poivron rouge': '1', 'oignon': '1', 'persil': '1c.à.s', 'moutarde': '2c.à.s', 'tabasco': '1trait', 'poivre': '', 'sel': '', 'mayonnaise': '3c.à.s', 'sauce anglaise': '1c.à.s'}), ('Sauce légère et relevée pour steak tartare', {'moutarde de Dijon': '1c.à.s', 'fromage blanc': '6cuillères', 'poivre': '', 'sel': '', 'câpres': '1c.à.c', 'ciboulette': '', 'échalote': '1', 'mayonnaise': ''}), ("Steak tartare à l'italienne : tomate séchée 

In [207]:
def convert(dico):
    for x,y in dico.items():
        for k,v in y.items():
            
            if v != '':
                n=int(("".join(re.findall('\d',str(v)))))
                
                l=str(''.join(z for z in v if not z.isdigit()))
                
                if l=='c.à.s rase':
                    y.update({k:str(15*n)+"g"})
                elif k=="jaune d'oeuf":
                    y.update({k:str(20*n)+"g"})
                elif k=="oeuf":
                    y.update({k:str(55*n)+"g"})
                elif l=='c.à.c':
                    y.update({k:str(5*n)+"g"})
                elif l=='c.à.s':
                    y.update({k:str(15*n)+"g"})
            elif v=="" and k=="sel" or k=="poivre":
                y.update({k:str(0.5)+"g"})
            
                
            
            

In [208]:
convert(test)
test

{'Steak tartare': {'moutarde de Dijon': '5g',
  'oignon': '15g',
  'câpres': '15g',
  'sauce worcestershire': '5g',
  'ketchup': '15g',
  'tabasco': '',
  'poivre': '0.5g',
  'sel': '0.5g',
  "huile d'olive": '30g',
  'persil': '5g',
  'boeuf': '250g',
  "jaune d'oeuf": '400g'},
 'Steak tartare arrangé': {'huile': '450g',
  'moutarde': '',
  'ketchup': '',
  'tabasco': '',
  'poivre': '0.5g',
  'sel': '0.5g',
  'câpres': '',
  'persil': '',
  'boeuf haché': '',
  'oeuf': '55g',
  'échalote': '1',
  'cornichons': ''},
 'Steak tartare végétarien': {'carotte': '500g',
  'câpres': '30g',
  'poivron rouge': '1',
  'oignon': '1',
  'persil': '15g',
  'moutarde': '30g',
  'tabasco': '1trait',
  'poivre': '0.5g',
  'sel': '0.5g',
  'mayonnaise': '45g',
  'sauce anglaise': '15g'},
 'Sauce légère et relevée pour steak tartare': {'moutarde de Dijon': '15g',
  'fromage blanc': '6cuillères',
  'poivre': '0.5g',
  'sel': '0.5g',
  'câpres': '5g',
  'ciboulette': '',
  'échalote': '1',
  'mayonnaise'

In [113]:
for x in test:
        for k,v in test.items():
            for x,y in v.items():
                print(y)
           
            

5
1c.à.s
1c.à.s
5
1c.à.s



2c.à.s
5
250g
1
450g








1
1

500g
2c.à.s
1
1
1c.à.s
2c.à.s
1trait


3c.à.s
1c.à.s
1c.à.s
6cuillères


5

1

250g
2c.à.s
1c.à.s
1c.à.s
20
7
4
50g



1
300g
5



2
2
150g
5
5
1⁄4
1petite gousse

1⁄4
1
1

20
20


4
4
16
4
300g
2c.à.s
2c.à.s
1c.à.s


1
1c.à.s
300g

1
1c.à.s
6



200g



600g
50g
1c.à.s
2
2
2
4
2cuillères
4
4

1c.à.s
7c.à.s


1pavé
1
1
150g
1⁄2
2
2
2tranches
2feuilles
5
5
1
5
2
1
2
1bouquet
3c.à.s



4
1bouquet
5g
5
1c.à.s
1c.à.s
5
1c.à.s



2c.à.s
5
250g
1
450g








1
1

500g
2c.à.s
1
1
1c.à.s
2c.à.s
1trait


3c.à.s
1c.à.s
1c.à.s
6cuillères


5

1

250g
2c.à.s
1c.à.s
1c.à.s
20
7
4
50g



1
300g
5



2
2
150g
5
5
1⁄4
1petite gousse

1⁄4
1
1

20
20


4
4
16
4
300g
2c.à.s
2c.à.s
1c.à.s


1
1c.à.s
300g

1
1c.à.s
6



200g



600g
50g
1c.à.s
2
2
2
4
2cuillères
4
4

1c.à.s
7c.à.s


1pavé
1
1
150g
1⁄2
2
2
2tranches
2feuilles
5
5
1
5
2
1
2
1bouquet
3c.à.s



4
1bouquet
5g
5
1c.à.s
1c.à.s
5
1c.à.s



2c.à.s
5
250g
1
450g








1
1

500g
2c.à.s


In [115]:
convert(test)

ValueError: too many values to unpack (expected 2)

In [103]:
test="erf456 efef56"
int(''.join(re.findall('\d',test)))



45656